In [63]:
import pennylane as qml
from pennylane import numpy as np
from pennylane.optimize import AdamOptimizer,NesterovMomentumOptimizer,AdagradOptimizer,RMSPropOptimizer
import pandas as pd
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import r2_score as r2
from sklearn.metrics import mean_absolute_error as mae
from sklearn.utils import shuffle
import strawberryfields as sf
from strawberryfields.ops import *
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn.model_selection import train_test_split
import time

In [64]:
dev = qml.device('strawberryfields.fock', wires=1, cutoff_dim=18,hbar=2)

In [65]:
def ansatz(v):
    qml.Rotation(v[0], wires=0)
    qml.Squeezing(v[1], 0, wires=0)
    qml.Rotation(v[2], wires=0)
    qml.Displacement(v[3], 0, wires=0)
    qml.Kerr(v[4], wires=0)

In [66]:
@qml.qnode(dev)
def qnn(var, x=None):
    
    # Encode input x into quantum state
    qml.DisplacedSqueezedState(x[0], 0,x[1],0, wires=0)

    for v in (var):
        ansatz(v)

    return qml.expval(qml.X(0))

In [67]:
@qml.qnode(dev)
def trace(var, x=None):
    
    qml.DisplacedSqueezedState(x[0],0,x[1],0,wires=0)
    
    for v in var:
    
        ansatz(v)
    
    return qml.expval(qml.Identity([0]))

In [68]:
def cost(var, features, labels):

    out = 0

    for i in range(len(features)):
        
        preds = qnn(var,x=features[i])
    
        out = out + (preds-labels[i])**2
    
    traces = [trace(var, x=x) for x in features]
    
    traces = (1-abs(np.array(traces))).mean()

    return (out/len(features))+0.01*traces

In [69]:

data = pd.read_csv('data.csv')

features = data.drop('I', axis=1).values
I = data.I.values

I = data.I.values*1000

X_train, X_test, y_train, y_test = train_test_split(features,I,
                                                    test_size=0.15,random_state=0)

Volt = X_train[:,0:1]
Inten = X_train[:,1:]

disp_scale = MinMaxScaler(feature_range=(-1.1,1)).fit(Volt)

squeeze_scale = MinMaxScaler(feature_range=(0,0.8)).fit(Inten)

X_train[:,0:1] = disp_scale.transform(X_train[:,0:1])
X_train[:,1:] = squeeze_scale.transform(X_train[:,1:])

X_test[:,0:1] = disp_scale.transform(X_test[:,0:1])
X_test[:,1:] = squeeze_scale.transform(X_test[:,1:])

X_train[5],y_train[5],abs(I).min()

(array([0.90869565, 0.8       ]), 1.02944, 0.000407583)

In [70]:
def accuracy(labels, predictions):
    score = mse(labels,predictions)
    s = mae(labels,predictions)
    r =r2(labels,predictions)
    return score,s,r

In [71]:
np.random.seed(1220)
var_init = np.random.normal(0,0.001,(8,5))
var = var_init
opt = AdamOptimizer(0.01, beta1=0.9, beta2=0.999)
opt.reset()

In [72]:

def testing(var,features):
    result = []
    for i in range(len(features)):
        preds = qnn(var,x=features[i])
        result.append(preds)
    return np.array(result).reshape(-1,1)

def next_batch(X, y, batchSize):
    rem = len(X)//batchSize
    out = len(X)%batchSize
    # loop over our dataset `X` in mini-batches of size `batchSize`
    for id ,i in enumerate(np.arange(0, X.shape[0], batchSize)):
        # yield a tuple of the current batched data and labels
        if id == rem and out != 0:
            temp = abs(batchSize-out)
            d = X[i:i + batchSize]
            l = y[i:i + batchSize]
            batch = np.random.choice(622,temp,replace=False)
            
            dd,ll = X[batch],y[batch]

            return (np.vstack((d,dd)),np.vstack((l.reshape(-1,1),ll.reshape(-1,1))))
        yield (X[i:i + batchSize], y[i:i + batchSize])

In [36]:

lossHistory = []
RR=[]
R = []
lossHistory_v = []
RR_v=[]
R_v = []
epoch_v = []
W = []
weight_epoch=[]
train = []
test = []
e = []    
mse_t = []
mse_v = []
tr=[]

old_acc = - np.inf
update_var = var
var_init

for it in range(10700):
    
    now = time.time()
    batch_test = np.random.choice(X_test.shape[0],X_test.shape[0],replace=False)
    X_test1 = X_test[batch_test]
    y_test1 = y_test[batch_test]
  
    batch = np.random.choice(X_train.shape[0],X_train.shape[0],replace=False)
    feats_train_11 =X_train[batch]
    Y_train1 = y_train[batch]
    
 #   feats_train_11,Y_train1 = shuffle(X_train,y_train)
    for id,(batchX, batchY) in enumerate(next_batch(feats_train_11, Y_train1,32)):
        
        var = opt.step(lambda v: cost(v, batchX, batchY), var)
        e.append(var)
        if id == 1:
            break
        
    if it > 100:
        np.save('weights9',np.array(e))    
        


    t=trace(var,x=X_train[5])
    print("Trace: {:0.5f}".format(t))   
    
    predictions = testing(var,X_test1)
    ww = accuracy(y_test1, predictions)
    acc_test = ww[2]
    acc_m = ww[0]
    weight_epoch.append(var)
    np.save('weights_per_epoch',np.array(weight_epoch))

    test.append(acc_test)
    np.save('testing',test)
    mse_v.append(acc_m)
    
    if acc_test > old_acc:
        update_var = var
        old_acc = acc_test
        print("  ")
        print('model improved')
        print("  ")
    
    print("Iter:{:5d} | mse: {:0.18e} | mae: {:0.18e} | R2: {:0.6f}".format(it + 1,ww[0],ww[1],ww[2]),'time:', int((time.time()-now)))

Trace: 0.99985
  
model improved
  
Iter:    1 | mse: 7.913816877857398069e-01 | mae: 7.493045107526957738e-01 | R2: -11.136052 time: 69
Trace: 0.99984
  
model improved
  
Iter:    2 | mse: 2.401651741540786478e-01 | mae: 4.227998323423743376e-01 | R2: -2.682998 time: 69
Trace: 0.99983
  
model improved
  
Iter:    3 | mse: 8.588933247858135389e-02 | mae: 2.098582110524263022e-01 | R2: -0.317136 time: 68
Trace: 0.99982
Iter:    4 | mse: 2.200846086380871536e-01 | mae: 2.902348279083872828e-01 | R2: -2.375057 time: 69
Trace: 0.99981
Iter:    5 | mse: 2.533144806671466798e-01 | mae: 3.130112787862571744e-01 | R2: -2.884646 time: 68
Trace: 0.99980
Iter:    6 | mse: 1.703939728359878947e-01 | mae: 2.654803193827595664e-01 | R2: -1.613038 time: 68
Trace: 0.99979
  
model improved
  
Iter:    7 | mse: 7.717312647756653654e-02 | mae: 2.131289185892355498e-01 | R2: -0.183471 time: 67
Trace: 0.99979
Iter:    8 | mse: 7.789295413100132126e-02 | mae: 2.417357193580499941e-01 | R2: -0.194510 time

KeyboardInterrupt: ignored

In [0]:


print(sf.hbar)
prog = sf.Program(1)

weight = np.load('./downloads/weights_per_epoch.npy')[363]

def layer(v):

    Rgate(v[0]) | q[0]
    Sgate(v[1]) | q[0]
    Rgate(v[2]) | q[0]
    Dgate(v[3]) | q[0]
    Kgate(v[4]) | q[0]

# sf.hbar = 2
# with prog.context as q:
#     Sgate(r=0.8) | q[0]
#     Dgate(a=-1.)| q[0]
# #    DisplacedSqueezed(alpha=-0.5,r=0.7)
# #    for x in weight:
#  #       layer(x)
# sf.hbar = 2
# eng = sf.Engine('fock',backend_options={'cutoff_dim':18})
# state = eng.run(prog).state
# prog.draw_circuit()

# X = np.linspace(-7.5, 7.5, 350)
# P = np.linspace(-7.5, 7.5, 350)
# Z = state.wigner(0, X, P)
# X, P = np.meshgrid(X, P)


# fig = plt.figure()
# ax = fig.add_subplot(2,1,1, projection="3d")

# surf = ax.plot_surface(X, P, Z, cmap='RdYlBu')
# fig.colorbar(surf)
# #ax.plot_surface(X, P, Z, cmap="RdYlBu", lw=0.5, rstride=1, cstride=1)
# #fig.set_size_inches(4.8, 5)
# ax.xaxis.set_pane_color((1.0, 1.0, 1.0, 1.0))
# ax.yaxis.set_pane_color((1.0, 1.0, 1.0, 1.0))
# ax.zaxis.set_pane_color((1.0, 1.0, 1.0, 1.0))
# # make the grid lines transparent
# ax.xaxis._axinfo["grid"]['color'] =  (1,1,1,1)
# ax.yaxis._axinfo["grid"]['color'] =  (1,1,1,1)
# ax.zaxis._axinfo["grid"]['color'] =  (1,1,1,1)

# ax.w_xaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
# ax.w_yaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
# ax.w_zaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))

# # Get rid of the spines
# ax.w_xaxis.line.set_color((1.0, 1.0, 1.0, 0.0))
# ax.w_yaxis.line.set_color((1.0, 1.0, 1.0, 0.0))
# ax.w_zaxis.line.set_color((1.0, 1.0, 1.0, 0.0))

# # Get rid of the ticks
# ax.set_xticks([])
# ax.set_yticks([])
# ax.set_zticks([])


# # ax.set_zlabel('Z')
# ax.set_xlabel('X')
# ax.set_ylabel('P')

# # ax.set_axis_off()
# plt.gca().axes.get_yaxis().set_visible(False)


prog = sf.Program(1)
with prog.context as q:
    Sgate(r=0.8) | q[0]
    Dgate(a=-1.)| q[0]
#    DisplacedSqueezed(alpha=-0.5,r=0.7)
    for x in weight:
        layer(x)
sf.hbar = 2
eng = sf.Engine('fock',backend_options={'cutoff_dim':18})
state = eng.run(prog).state

# # fig = plt.figure()
# X = np.linspace(-7.5, 7.5, 350)
# P = np.linspace(-7.5, 7.5, 350)
# Z = state.wigner(0, X, P)
# X, P = np.meshgrid(X, P)



# ax = fig.add_subplot(2,1,2, projection="3d")

# surf = ax.plot_surface(X, P, Z, cmap='RdYlBu')
# fig.colorbar(surf)
# ax.xaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
# ax.yaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
# ax.zaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
# # make the grid lines transparent
# ax.xaxis._axinfo["grid"]['color'] =  (1,1,1,0)
# ax.yaxis._axinfo["grid"]['color'] =  (1,1,1,0)
# ax.zaxis._axinfo["grid"]['color'] =  (1,1,1,0)

# ax.w_xaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
# ax.w_yaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
# ax.w_zaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))

# # Get rid of the spines
# ax.w_xaxis.line.set_color((1.0, 1.0, 1.0, 0.0))
# ax.w_yaxis.line.set_color((1.0, 1.0, 1.0, 0.0))
# ax.w_zaxis.line.set_color((1.0, 1.0, 1.0, 0.0))

# # Get rid of the ticks
# ax.set_xticks([])
# ax.set_yticks([])
# ax.set_zticks([])


# # ax.set_zlabel('Z')
# ax.set_xlabel('X')
# ax.set_ylabel('P')
# #ax.plot_surface(X, P, Z, cmap="RdYlBu", lw=0.5, rstride=1, cstride=1)
# #fig.set_size_inches(4.8, 5)
# # ax.set_axis_off()